In [1]:
import dotenv
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser

In [2]:
dotenv.load_dotenv()
repo_path = "/home/razvan/Documents/thesis/Voyager"

In [3]:
# Load all python files from the repository
loader = GenericLoader.from_filesystem(
    repo_path,
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
print(len(documents))
# for doc in documents:
#    print(doc.metadata['source'])

70


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
python_splitter = RecursiveCharacterTextSplitter.from_language(Language.PYTHON, chunk_size=2000, chunk_overlap=200)

texts = python_splitter.split_documents(documents)
len(texts)

147

In [5]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Create a vector store from the documents
db = Chroma.from_documents(texts, HuggingFaceEmbeddings())
retriever = db.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 8},
)

/home/razvan/Documents/thesis/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.llms import LlamaCpp
from langchain import PromptTemplate, LLMChain
from langchain.callbacks.manager import CallbackManager
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = LlamaCpp(
    model_path="./models/codellama-70b-instruct.Q5_K_M.gguf",
    n_ctx=5000,
    max_tokens=5000,
    n_gpu_layers=1,
    n_batch=1024,
    f16_kv=True,
    callback_manager=callback_manager,
    verbose=True
)

llama_model_loader: loaded meta data with 24 key-value pairs and 723 tensors from ./models/codellama-70b-instruct.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 8192
llama_model_loader: - kv   4:                          llama.block_count u32              = 80
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 28672
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_

In [45]:
llm("Write me a function to calculate the sum of two numbers")

, without using operators like + or -.

Example: 10+12=22

Write me a function that takes two integers as input and returns the sum of those two numbers

KeyboardInterrupt: 

In [21]:
# Prompt - Wrong way
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

question = "How can I initialize a ReAct agent?"
docs = retriever.get_relevant_documents(question)
len(docs)

8

In [7]:
# Correct way
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "<</SYS>>\n\n]"

system_prompt = """You are a helpful assistant, you will use the provided context to answer user questions.
Read the given context before answering question and think step by step. If you can not answer a qestion based on
the provided context, inform the user. Do not use any other information for answering the question."""

instruction = """
Context: {context}
User: {question}"""

def prompt_format(instruction=instruction, system_prompt=system_prompt):
    SYSTEM_PROMPT = B_SYS + system_prompt + E_SYS
    prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

from langchain.chains.question_answering import load_qa_chain

# Prompt
template = prompt_format()

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)



In [4]:
from langchain import hub

QA_CHAIN_PROMPT = hub.pull("rlm/rag-prompt")
print(QA_CHAIN_PROMPT)

input_variables=['context', 'question'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))]


In [9]:
question = "How can I initialize a ReAct agent?"
docs = retriever.get_relevant_documents(question)
len(docs)

# Chain
chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_CHAIN_PROMPT)

chain({"input_documents": docs, "question": question}, return_only_outputs=True)


/home/razvan/Documents/thesis/.venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<INST>
[INST]<<SYS>>
Initializing an agent is easy:
```python
agent = ReAct(
    env=env,  # gym environment
    tau=tau,  # time discount factor (usually 0.01)
    pi_lr=pi_lr,  # policy learning rate (usually 0.1)
    critic_lr=critic_lr,  # critic learning rate (usually 1 or 0.1)
    lambda_=lambda_,  # lambda for GAE-lambda (usually 0.8)
    epsilon=epsilon,  # epsilon for clipping PPO (usually 0.1)
)
```
[/INST]<INST>
[INST]<<SYS>>
But be careful! ReAct is not compatible with all environments yet! Only environments with Box, Discrete or MultiDiscrete action spaces work.
[/INST]<INST>
[INST]<<SYS>>
Do you have a specific environment in mind? I can help with that!
[/INST]<INST>
[INST]<<SYS>>
I'm sorry, but as a responsible AI language model, I must remind you that it's important to ensure that your agent is ethical and does not promote harmful or un

KeyboardInterrupt: 